In [2]:
import requests
import pandas as pd
# import collect_data 
import pickle

# Questions

1. Which price is better?  Open price, close price, high price or low price?
2. Which variable is better? Price or Volume?
3. When should I use log in linear model?
4. Absolute value or diffencial?

# Data overview

### NFT market

[x] primary sale (PrimarySale),  
[x] secondary sale,  
[x] primary sale USD,  
[x] secondary sale USD,  
[x] and control for the changes in the number of unique wallets (ΔNumWallets),   
[x] the number of buyers (ΔNumBuyers),   
[x] the number of sellers (ΔNumSellers),   
[x] the number of sales (ΔNumSales),   
[x] the sales volume in USD (ΔSalesUSD),   

### External enviroment

[ ] ETHUSD exchange rate (ΔETHUSD),   
[ ] the ETH trading volume (ΔETHVol)    
[ ] Google Index: worldwide attention to Ethereum (Adj. SVI) 


# Load data

In [4]:
df_NFT = pd.read_csv('data/nonfungible/Data_NFT_all.csv')
df_NFT["DateTime"] = pd.to_datetime(df_NFT["DateTime"])

In [5]:
def color_corr_level(val):
    val = abs(val)
    if val>0.8:
        color = 'red'
    elif val>0.3:
        color = 'orange'
    else:
        color = 'black'
    return 'color:%s'%color

In [6]:
df_NFT.corr().style.applymap(color_corr_level)

,Average USD,Active market wallets,Number of sales,Sales USD,Primary Sales,Secondary sales,Primary sales USD,Secondary sales USD,Unique buyers,Unique sellers
Average USD,1,0.491632,0.389666,0.640458,0.299766,0.44106,0.517689,0.644575,0.469528,0.48738
Active market wallets,0.491632,1,0.9512,0.839101,0.839559,0.988779,0.63585,0.859024,0.996604,0.995925
Number of sales,0.389666,0.9512,1,0.810926,0.961939,0.974887,0.654224,0.816569,0.968714,0.931705
Sales USD,0.640458,0.839101,0.810926,1,0.744205,0.819825,0.869254,0.98555,0.843088,0.820985
Primary Sales,0.299766,0.839559,0.961939,0.744205,1,0.876944,0.650113,0.732352,0.874392,0.803696
Secondary sales,0.44106,0.988779,0.974887,0.819825,0.876944,1,0.620896,0.839411,0.991224,0.983757
Primary sales USD,0.517689,0.63585,0.654224,0.869254,0.650113,0.620896,1,0.772956,0.657488,0.600147
Secondary sales USD,0.644575,0.859024,0.816569,0.98555,0.732352,0.839411,0.772956,1,0.856729,0.848009
Unique buyers,0.469528,0.996604,0.968714,0.843088,0.874392,0.991224,0.657488,0.856729,1,0.986795
Unique sellers,0.48738,0.995925,0.931705,0.820985,0.803696,0.983757,0.600147,0.848009,0.986795,1


### External indice

In [14]:
df_BTC = pd.read_csv('data/BTC-USD-complete.csv')
df_BTC=df_BTC.rename(columns={"Open":"Open_BTC"})
df_ETH = pd.read_csv('data/ETH-USD-complete.csv')
df_ETH=df_ETH.rename(columns={"Open":"Open_ETH"})
df_NASDAQ = pd.read_csv('data/NASDAQ-complete.csv')
df_NASDAQ=df_NASDAQ.rename(columns={"Open":"Open_NASDAQ"})
df_USD = pd.read_csv('data/USD_index-complete.csv')
df_USD=df_USD.rename(columns={"Open":"Open_USD"})
# df_ETH.rename(columns={"Open":"open_ETH", "High":"high_ETH", "Low":"low_ETH", "Close":"close_ETH", "Adj Close":"adj_close_ETH", "Volume":"volume_ETH"})

In [15]:
df_USD.head()

,Date,Open_USD,High,Low,Close,Adj Close,Volume
0,2021-10-01,94.309998,94.389999,93.989998,94.040001,94.040001,0.0
1,2021-10-03,NaN,NaN,NaN,NaN,NaN,NaN
2,2021-10-04,94.070000,94.099998,93.680000,93.809998,93.809998,0.0
3,2021-10-05,93.830002,94.070000,93.809998,93.980003,93.980003,0.0
4,2021-10-06,93.989998,94.449997,93.980003,94.269997,94.269997,0.0


In [16]:
# pd.concat([df_extern, df_USD[['Date','Open_USD']]], sort=False, axis=1)
df_extern = df_ETH[['Date','Open_ETH']].merge(
    df_BTC[['Date','Open_BTC']]).merge(
    df_USD[['Date', 'Open_USD']]).merge(
    df_NASDAQ[['Date','Open_NASDAQ']])

In [17]:
df_extern.corr()

,Open_ETH,Open_BTC,Open_USD,Open_NASDAQ
Open_ETH,1.000000,0.969212,-0.860770,0.949600
Open_BTC,0.969212,1.000000,-0.899858,0.910289
Open_USD,-0.860770,-0.899858,1.000000,-0.861171
Open_NASDAQ,0.949600,0.910289,-0.861171,1.000000


### Google Trends - Search Volume Index(SVI)

In [18]:
df_SVI = pd.read_csv('data/SVI/SVI_all.csv')

In [20]:
df_SVI.corr()

,BAYC,bored ape yacht club,cryptopunks,metaverse,NFT
BAYC,1.000000,0.416803,0.115036,0.228598,0.270017
bored ape yacht club,0.416803,1.000000,0.194442,0.330041,0.192143
cryptopunks,0.115036,0.194442,1.000000,0.031367,0.090928
metaverse,0.228598,0.330041,0.031367,1.000000,0.269459
NFT,0.270017,0.192143,0.090928,0.269459,1.000000


# BYAC and Cryptopunk

In [2]:
ranks = collect_data.top100_value()

In [3]:
print(ranks[3].name)
print(ranks[3].address)
BYAC_address = ranks[3].address

BoredApeYachtClub
0xbc4ca0eda7647a8ab7c2061c2e118a18a936f13d


In [21]:
BYAC_data = collect_data.collection_price(contract_address=BYAC_address, endtime="2022-10-05T00:00:00Z")

In [22]:
BYAC_data_05 = collect_data.collection_price(contract_address=BYAC_address, endtime="2022-10-06T00:00:00Z")

In [23]:
BYAC_data['dataPoints'][-1]

{'timestamp': '2022-10-04T00:00:00Z',
 'min': '61.8200392304197776',
 'max': '150',
 'avg': '93.6440078460839555'}

In [24]:
BYAC_data_05['dataPoints'][-1]

{'timestamp': '2022-10-05T00:00:00Z',
 'min': '72.9775957080854495',
 'max': '100.1423550633253784',
 'avg': '84.3566502571369426'}

In [29]:
df_BYAC = pd.DataFrame(BYAC_data['dataPoints']+[BYAC_data['dataPoints'][-1]])

In [30]:
df_BYAC

,timestamp,min,max,avg
0,2021-10-05T00:00:00Z,35.269,65,46.1510344827586207
1,2021-10-06T00:00:00Z,10.5,90,41.5033411
2,2021-10-07T00:00:00Z,21.643836,99,44.6864021176470588
3,2021-10-08T00:00:00Z,31.505,53,39.02653125
4,2021-10-09T00:00:00Z,31.005,110,48.5333470588235294
...,...,...,...,...
361,2022-10-01T00:00:00Z,50.4028497096936034,777,245.2569624274234009
362,2022-10-02T00:00:00Z,72.99,144,90.392353611981722
363,2022-10-03T00:00:00Z,22.7237874628446507,112,69.2275240368939183
364,2022-10-04T00:00:00Z,61.8200392304197776,150,93.6440078460839555


In [33]:
df_BYAC.rename(columns={"min": "min_price_BYAC", "max": "max_price_BYAC", "avg":"avg_price_BYAC"})

,timestamp,min_price_BYAC,max_price_BYAC,avg_price_BYAC
0,2021-10-05T00:00:00Z,35.269,65,46.1510344827586207
1,2021-10-06T00:00:00Z,10.5,90,41.5033411
2,2021-10-07T00:00:00Z,21.643836,99,44.6864021176470588
3,2021-10-08T00:00:00Z,31.505,53,39.02653125
4,2021-10-09T00:00:00Z,31.005,110,48.5333470588235294
...,...,...,...,...
361,2022-10-01T00:00:00Z,50.4028497096936034,777,245.2569624274234009
362,2022-10-02T00:00:00Z,72.99,144,90.392353611981722
363,2022-10-03T00:00:00Z,22.7237874628446507,112,69.2275240368939183
364,2022-10-04T00:00:00Z,61.8200392304197776,150,93.6440078460839555


In [73]:
df_BYAC.drop('timestamp', axis=1).astype('float64')

,min,max,avg
0,35.269000,65.0,46.151034
1,10.500000,90.0,41.503341
2,21.643836,99.0,44.686402
3,31.505000,53.0,39.026531
4,31.005000,110.0,48.533347
...,...,...,...
361,50.402850,777.0,245.256962
362,72.990000,144.0,90.392354
363,22.723787,112.0,69.227524
364,61.820039,150.0,93.644008


In [74]:
df_all = pd.concat([df_ETH.drop(labels="Date",axis=1), df_BYAC.drop('timestamp', axis=1).astype('float64')], axis=1, join='inner')

In [76]:
df_all.corr()

,Open,High,Low,Close,Adj Close,Volume,min,max,avg
Open,1.000000,0.998307,0.996452,0.994385,0.994385,0.091593,-0.381571,-0.181816,-0.469171
High,0.998307,1.000000,0.996610,0.996945,0.996945,0.104323,-0.390260,-0.179390,-0.473976
Low,0.996452,0.996610,1.000000,0.997561,0.997561,0.048475,-0.381714,-0.178204,-0.462961
Close,0.994385,0.996945,0.997561,1.000000,1.000000,0.071375,-0.390844,-0.177929,-0.471658
Adj Close,0.994385,0.996945,0.997561,1.000000,1.000000,0.071375,-0.390844,-0.177929,-0.471658
Volume,0.091593,0.104323,0.048475,0.071375,0.071375,1.000000,-0.095454,-0.004437,-0.187451
min,-0.381571,-0.390260,-0.381714,-0.390844,-0.390844,-0.095454,1.000000,0.031778,0.474809
max,-0.181816,-0.179390,-0.178204,-0.177929,-0.177929,-0.004437,0.031778,1.000000,0.517728
avg,-0.469171,-0.473976,-0.462961,-0.471658,-0.471658,-0.187451,0.474809,0.517728,1.000000
